In [0]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [0]:
# to import the files

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/jigsaw-toxic-comment-classification-challenge

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/jigsaw-toxic-comment-classification-challenge


In [0]:
#read the data
training=pd.read_csv('/gdrive/My Drive/jigsaw-toxic-comment-classification-challenge/train.csv')
testing=pd.read_csv('/gdrive/My Drive/jigsaw-toxic-comment-classification-challenge/test.csv')

In [0]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = training[list_classes].values
print(y)
list_sentences_train = training["comment_text"]
list_sentences_test = testing["comment_text"]

list_sentences_test[5]

[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 ...
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


'Thank you for understanding. I think very highly of you and would not revert without discussion.'

In [0]:
list_sentences_train[5]
print(training.head())
data = training.values
X1_train = data[:,1]
Y_train = data[:,2]
print(Y_train)
print(X1_train)

                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0  
[0 0 0 ... 0 0 0]
["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I

In [0]:
# Tokenization
#https://machinelearningmastery.com/prepare-text-data-deep-learning-keras/
#max_features = 20000
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X1_train)
encoded = tokenizer.texts_to_sequences(X1_train)

print(X1_train[8])
print(encoded[8:9])

Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169
[[263, 22, 1, 286, 34847, 24, 1476, 2, 6, 489, 73, 14, 9215, 2, 330, 215, 10, 1, 23, 1473, 51, 47, 2898, 15, 35, 12, 199, 73, 1051, 1060, 9, 11, 16, 57, 1390, 37, 48, 40, 81, 11, 12, 480, 17, 5, 273, 7, 18, 55, 2, 1, 4850, 8153, 27, 26, 78, 669, 5, 1368, 11, 687, 2, 96147, 96148, 53, 8, 5, 702, 7774, 23, 9, 1094, 6, 46, 468, 41, 251, 16, 159, 324, 21, 3566, 10, 13824, 3439, 4584, 3890, 4851]]


In [0]:
# Finding the maximum lenght for padding
max_len = 0;
for i in range(0, len(encoded)):
    if max_len < len(encoded[i]):
        max_len = len(encoded[i])

max_len = max_len + 1
print(max_len)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

1404
210338


In [0]:
#in padding, by default pre padding
#we are going for post, why? ask Rishi
from sklearn.model_selection import train_test_split
padded_docs = pad_sequences(encoded, maxlen=max_len, padding='post')
print(padded_docs)
#X_train, X_test, Y_train, Y_test = train_test_split( padded_docs, Y_train, test_size=0.2, random_state=42)

[[  688    75     1 ...     0     0     0]
 [96145    52  2635 ...     0     0     0]
 [  412   437    73 ...     0     0     0]
 ...
 [34278  7330  5208 ...     0     0     0]
 [    4    11   574 ...     0     0     0]
 [    4     7   134 ...     0     0     0]]


In [0]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
#X_train, Y_train = ros.fit_resample(padded_docs, Y_train)

#X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

X_train, X_test, Y_train, Y_test = train_test_split( padded_docs, Y_train, test_size=0.2, random_state=42)
X_train, Y_train = ros.fit_resample(X_train, Y_train)


/usr/local/lib/python3.6/dist-packages/imblearn/utils/_validation.py:93: UserWarning: 'y' should be of types ('binary', 'multiclass', 'multilabel-indicator') only. Got unknown instead.
  TARGET_KIND, type_of_target(y)))


In [0]:
embeddings_index = dict()
f = open('/gdrive/My Drive/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()

In [0]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [0]:
#defining the input layer
input_seq = Input(shape=(1404, ))
#here 1404 is maxlenght of padding , so the size of our input

In [0]:
embed = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=True)(input_seq)
#defining the embedding layer and passing the imput throught it


Instructions for updating:
Colocations handled automatically by placer.


In [0]:
embed2 = LSTM(60, return_sequences=True,name='lstm_layer1')(embed)

In [0]:
x1 = GlobalMaxPool1D()(embed2)

In [0]:
x1 = Dense(50, activation="relu")(x1)

In [0]:
#https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/
x1 = Dropout(0.1)(x1)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
x1 = Dense(1, activation="sigmoid")(x1)


In [0]:
classifier = Model(input_seq, x1)
classifier.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
print(classifier.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1404)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1404, 100)         21033800  
_________________________________________________________________
lstm_layer1 (LSTM)           (None, 1404, 60)          38640     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 60)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                3050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total para

In [0]:
print(X_train)
print(X_test)

[[14832  9031    56 ...     0     0     0]
 [  674    85   819 ...     0     0     0]
 [    1  5487     3 ...     0     0     0]
 ...
 [   62  2631   129 ...     0     0     0]
 [    7    44 58611 ...     0     0     0]
 [  847   107    35 ...     0     0     0]]
[[13327    19     6 ...     0     0     0]
 [34915  1411    94 ...     0     0     0]
 [ 4070    46  4504 ...     0     0     0]
 ...
 [  345 63107   277 ...     0     0     0]
 [    5  1684   576 ...     0     0     0]
 [28891    89    20 ...     0     0     0]]


In [0]:
#verbose: Integer. 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
#Not used , validation_split: Float between 0 and 1. Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. The validation data is selected from the last samples in the x and y data provided, before shuffling.
classifier.fit(X_train, Y_train, batch_size=512, epochs = 10, 
    verbose=1, validation_data= (X_test, Y_test))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 230836 samples, validate on 31915 samples
Epoch 1/10
230836/230836 [==============================] - 1056s 5ms/step - loss: 0.2965 - acc: 0.8707 - val_loss: 0.1813 - val_acc: 0.9306
Epoch 2/10
230836/230836 [==============================] - 1049s 5ms/step - loss: 0.1978 - acc: 0.9197 - val_loss: 0.2743 - val_acc: 0.8869
Epoch 3/10
230836/230836 [==============================] - 1078s 5ms/step - loss: 0.1681 - acc: 0.9334 - val_loss: 0.1684 - val_acc: 0.9298
Epoch 4/10
230836/230836 [==============================] - 1058s 5ms/step - loss: 0.1461 - acc: 0.9442 - val_loss: 0.1427 - val_acc: 0.9413
Epoch 5/10
230836/230836 [==============================] - 1050s 5ms/step - loss: 0.1277 - acc: 0.9532 - val_loss: 0.1565 - val_acc: 0.9355
Epoch 6/10
230836/230836 [==============================] - 1052s 5ms/step - loss: 0.1093 - acc: 0.9620 - val_loss: 0.

In [0]:
classifier.save('./GLOVE_CNN.h5')
classifier.save("./G_further.h5")
from keras.models import load_model

In [0]:
predictions = classifier.predict(X_test)
print(predictions)

for i in range(len(predictions)):
    if predictions[i]>0.5:
        predictions[i] = 1
    else:
        predictions[i] = 0

print(predictions)

[[2.0175561e-01]
 [3.2782555e-07]
 [8.8512897e-06]
 ...
 [1.0229576e-04]
 [9.9825972e-01]
 [8.9111512e-05]]
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [1.]
 [0.]]


In [0]:
from sklearn.metrics import confusion_matrix, classification_report

print(predictions)
Y_test = [[i] for i in Y_test]

print(Y_test)
print(classification_report(Y_test, predictions))

pd.DataFrame(
   confusion_matrix(Y_test, predictions),
   index = [['Actual', 'Actual'], ['Not Toxic', 'Toxic']],
   columns = [['Predicted', 'Predicted'], ['Not Toxic', 'Toxic']])

[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [1.]
 [0.]]
[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [1], [1], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0

Predicted      
                 Not Toxic Toxic
Actual Not Toxic     27583  1276
       Toxic           438  2618